<a href="https://colab.research.google.com/github/faranbuttedu/Valencia-Flood-Mapping/blob/main/Mapping_Valencia_Flood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environmental Risk Analysis using Sentinel-1 SAR: Mapping the October 2024 Valencia Flood**

**Abstract**

This study utilizes Synthetic Aperture Radar (SAR) imagery from the Sentinel-1 mission, processed on the Google Earth Engine (GEE) platform, to map the devastating flood extent in the Valencia region, Spain, following the catastrophic DANA (Cold Drop) event of October 2024. The objective is to apply a robust change detection methodology to delineate the inundated areas, distinguishing newly flooded land from permanent water bodies. The methodology involves collecting pre- and post-flood C-band VV-polarized Sentinel-1 Ground Range Detected (GRD) data, applying speckle filtering, and calculating a backscatter intensity difference. A key refinement step involves masking out permanent water areas using the JRC Global Surface Water dataset to generate a flood mask exclusively over land. Results include visual maps of the before-and-after conditions and an estimated total area of inundation in square kilometers, providing essential geospatial data for disaster assessment and management.

**Introduction**

**Presentation of the Environmental Phenomenon**

In late October 2024, Spain was struck by one of the deadliest natural disasters in its modern history when an intense meteorological phenomenon, known locally as a DANA or Gota Fría (cold drop), brought torrential rainfall to the eastern and southern regions. The Valencian Community, particularly the peripheral municipalities in the Horta Sud and Camp de Túria regions, faced catastrophic flash flooding due to rainfall exceeding annual averages in just a few hours. The event, exacerbated by rapid runoff from steep topography and the urbanization of coastal floodplains, led to widespread infrastructural damage and significant loss of life.

**Scientific and Environmental Relevance**

Accurate and timely mapping of flood extent is crucial for rescue operations, damage assessment, and future risk mitigation strategies. Traditional post-flood surveys are often hampered by inaccessible areas. Therefore, Geographic Information Systems (GIS) and remote sensing techniques provide the necessary tools for rapid, large-area coverage. This analysis focuses on mapping the maximum inundation extent to quantify the disaster's physical footprint.

**Sentinel-1 SAR Data**

The Copernicus Sentinel-1 mission, comprising two satellites (1A and 1B), is ideal for flood monitoring because it uses Synthetic Aperture Radar (SAR), an active microwave sensor.

**Function of Sensors:**

 SAR transmits its own signal and records the backscatter—the energy reflected back to the sensor.

**Flood Detection Principle:**

Smooth water surfaces act like mirrors, causing the C-band radar signal to reflect away from the sensor (specular reflection), resulting in a very low backscatter value (dark pixels) in the image. Conversely, land, vegetation, and urban areas cause high backscatter due to diffuse reflection (bright pixels).

**Advantage:**

Unlike optical sensors (like Sentinel-2), SAR can penetrate clouds and operate day and night, making it indispensable during flood events which are typically characterized by heavy rain and cloud cover. The analysis uses VV (Vertical transmit/Vertical receive) polarization, which is sensitive to surface water changes.

**Materials and Methods**


**Methodology and Data Processing**

Flood mapping using Sentinel-1 typically relies on a change detection approach, comparing the backscatter intensity of a pre-event image (reference) with a post-event image (flood observation). A significant decrease in backscatter in the post-event image indicates the presence of new water (flood) on what was previously land. All processing is executed on the Google Earth Engine (GEE) cloud computing platform, accessed via the Python API and the geemap library, using the user's project ID for authentication.

**Code Cell 1: Initialization and Parameters**

In [ ]:
# --- CODE CELL 1: Initialization and Parameters (Materials & Methods) ---
import ee
import geemap
import math # Needed for exploratory analysis

# --- 1. Earth Engine Initialization ---
# Authenticate Earth Engine to ensure access to data
ee.Authenticate()

try:
    # Initialize Earth Engine with the specified project
    # Project ID provided by the user: ee-faranbuttedu
    ee.Initialize(project='ee-faranbuttedu')
    print("Earth Engine initialized successfully.")
except Exception as e:
    # Fallback for GEE login issue in some environments
    print(f"Error during GEE initialization: {e}")
    try:
        ee.Initialize()
        print("Earth Engine initialized with default credentials.")
    except:
        print("Earth Engine initialization failed.")
        raise

# --- 2. Define Parameters and Area of Interest (AOI) ---

# Valencia Region, Spain (Approximate coordinates for the affected area)
# Defined as a rectangle covering major affected municipalities (e.g., Paiporta, Torrent, Chiva)
AOI = ee.Geometry.Rectangle([-0.7, 39.0, 1.2, 40.5])
CENTER_LON = 0.25
CENTER_LAT = 39.75
ZOOM_LEVEL = 9

# Dates for Sentinel-1 Imagery (October 2024 Flood)
# Pre-Flood: September 2024 (Dry period before the DANA event)
BEFORE_START = '2024-09-15'
BEFORE_END = '2024-09-30'

# Post-Flood: Immediately after the main event (Oct 29-30)
AFTER_START = '2024-10-30'
AFTER_END = '2024-11-15'

# Flood Detection Threshold: A negative value indicates a drop in backscatter (new water).
# This is a critical parameter where the threshold is set to -5 dB change (After - Before).
FLOOD_THRESHOLD = -5

print(f"AOI: Valencia Region, Spain | Study Period: {BEFORE_START} to {AFTER_END}")

Earth Engine initialized successfully.
AOI: Valencia Region, Spain | Study Period: 2024-09-15 to 2024-11-15


**Code Cell 2: Functions and Image Composites**

In [ ]:
# --- CODE CELL 2: Functions and Image Collection Setup (Materials & Methods) ---

def mask_edge(image):
    """
    Masks out image border artifacts, where backscatter values are extremely low.
    """
    # Masks values less than -30.0 dB
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# Filter Sentinel-1 Collection
s1_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(AOI)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW')) # Interferometric Wide Swath mode
    .select('VV')
    .map(mask_edge)
)

# Load JRC Global Surface Water Mapping (JRC GSW) permanent water mask
# Occurrence band > 50 means water is present over 50% of the time (Permanent Water)
JRC_GSW = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
water_occurrence = JRC_GSW.select('occurrence')
permanent_water_mask = water_occurrence.gt(50).clip(AOI)

# Before Flood Image: Mean of images from the 'BEFORE' period (Reference)
before_flood = (
    s1_collection.filterDate(BEFORE_START, BEFORE_END)
    .mean()
    .clip(AOI)
    .rename('before_VV')
)

# After Flood Image: Mean of images from the 'AFTER' period (Observation)
after_flood = (
    s1_collection.filterDate(AFTER_START, AFTER_END)
    .mean()
    .clip(AOI)
    .rename('after_VV')
)

print(f"Successfully filtered Sentinel-1 data for {BEFORE_START}-{AFTER_END}.")
print("Before- and After-Flood mean composites generated.")

Successfully filtered Sentinel-1 data for 2024-09-15-2024-11-15.
Before- and After-Flood mean composites generated.


**Results**

**Part 1: Basic Analysis and Visualization**

The core result is the Flood Mask, derived from the difference between the post-flood and pre-flood images. This change detection technique identifies pixels where the backscatter dropped below the defined threshold ($-5$ dB), indicating a change from land to water. Crucially, this mask is combined with the Permanent Water Mask (JRC GSW) to exclude existing river channels and the Mediterranean Sea, ensuring the final output highlights only the newly inundated land areas.

**Code Cell 3: Flood Mask Generation and Map Display**

In [ ]:
# --- CODE CELL 3: Flood Mask Generation and Map Display (Results, Part 1) ---

def calculate_flood_mask(before_img, after_img, water_mask, threshold):
    """
    Calculates the flood extent restricted to non-permanent water areas (land).
    """
    # Calculate difference (After - Before)
    difference = after_img.subtract(before_img)

    # Identify potential flood pixels: difference < threshold
    flood = difference.lt(threshold).rename('flood')

    # Restrict flood detection to land areas (where water_mask == 0)
    flood_on_land = flood.And(water_mask.eq(0))

    return flood_on_land.updateMask(flood_on_land) # Use as mask for visibility

# Calculate the final flood mask
flood_mask = calculate_flood_mask(
    before_flood,
    after_flood,
    permanent_water_mask,
    threshold=FLOOD_THRESHOLD
)

# Visualization Parameters
sar_vis_params = {
    'min': -20,
    'max': -5,
    'palette': ['000000', 'FFFFFF'] # Black (low backscatter/water) to White (high backscatter/land)
}
flood_vis_params = {
    'palette': ['0000FF'] # Solid Blue for the flood mask
}

# Map Setup and Layer Addition
m = geemap.Map(center=[CENTER_LAT, CENTER_LON], zoom=ZOOM_LEVEL)
m.setOptions('HYBRID')

# 1. Before Flood Layer (Baseline)
m.add_layer(
    before_flood,
    sar_vis_params,
    f'S1 VV Before Flood ({BEFORE_START} to {BEFORE_END})',
    False
)

# 2. After Flood Layer (Observation)
m.add_layer(
    after_flood,
    sar_vis_params,
    f'S1 VV After Flood ({AFTER_START} to {AFTER_END})',
    True
)

# 3. Flood Mask Layer (Thematic Result)
m.add_layer(
    flood_mask,
    flood_vis_params,
    '⚠️ October 2024 Flood Mask (Land Only)',
    True
)

# Display the map
print("Interactive map generated. View the map below to see the layers.")
m

Interactive map generated. View the map below to see the layers.


Map(center=[39.75, 0.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

**Part 2: Exploratory Analysis (Quantitative Area Estimation)**

As an exploratory analysis, the total area of the detected flood inundation is calculated in square kilometers ($\text{km}^2$). This quantitative measure provides an immediate estimate of the disaster's magnitude, which can be compared against known disaster reports (e.g., initial reports estimated approximately $199 \text{ km}^2$ were inundated). The GEE platform is used to compute the area by summing the pixel area of all pixels classified as flooded in the final flood_mask.

**Code Cell 4: Quantitative (Exploratory) Analysis**

In [ ]:
# --- CODE CELL 4: Quantitative (Exploratory) Analysis (Results, Part 2) ---

# 1. Calculate the area of each pixel within the flood mask.
# ee.Image.pixelArea() provides the area of each pixel in square meters.
# Masking ensures we only consider the area of flooded pixels.
pixel_area_m2 = ee.Image.pixelArea().mask(flood_mask)

# 2. Sum the area of all flooded pixels within the Area of Interest (AOI).
# scale=10 corresponds to the native resolution of Sentinel-1 GRD data.
flooded_area_m2 = pixel_area_m2.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI,
    scale=10,
    maxPixels=1e13 # Allow for large calculation
)

# 3. Get the value and convert from m^2 to km^2 (divide by 1,000,000)
# We use .getInfo() to trigger the computation and fetch the result from GEE.
# The result is stored as a dictionary, so we access the value using .get('area').
try:
    area_m2_value = ee.Number(flooded_area_m2.get('pixel_area')).getInfo() # Use 'pixel_area' key or adjust based on GEE output
    if area_m2_value is None:
        # Fallback for GEE naming convention for pixelArea sum
        area_m2_value = ee.Number(flooded_area_m2.get('sum')).getInfo()
except:
    # Use the area key if GEE names the summed band 'area' by default
    area_m2_value = ee.Number(flooded_area_m2.get('area')).getInfo()


# Final calculation and print
if area_m2_value is not None:
    flooded_area_sqkm = area_m2_value / 1e6
    print(f"--- Exploratory Analysis: Quantitative Flood Area Estimation ---")
    print(f"Total Flooded Area (m²): {area_m2_value:,.0f} m²")
    print(f"Estimated Flooded Area (km²): {flooded_area_sqkm:,.2f} km²")
else:
    print("Could not retrieve area calculation result from Earth Engine.")

--- Exploratory Analysis: Quantitative Flood Area Estimation ---
Total Flooded Area (m²): 61,488,436 m²
Estimated Flooded Area (km²): 61.49 km²


**Discussion**

**Critical Analysis of Sentinel-1 SAR Capacity**

The application of Sentinel-1 SAR imagery, processed via GEE, demonstrates a highly effective and near-real-time capacity for disaster response. The key advantage of SAR is its independence from atmospheric conditions, providing crucial data even under the heavy cloud cover that defined the DANA event. The change detection method using the VV channel is a robust, widely accepted technique for distinguishing temporary flooding from permanent water bodies.



**Limitations and Difficulties Encountered**

**Threshold Dependency:**

The use of a fixed threshold ($-5$ dB) is a primary limitation. The optimal threshold for identifying flood water can vary significantly based on local conditions, such as soil type, land cover (e.g., highly vegetated areas require a different threshold), and surface roughness. A single global threshold may lead to false positives (e.g., shadows or smooth tarmac misidentified as water) or false negatives (e.g., flooded areas with wind-roughened surfaces or vegetation cover omitted). More advanced methodologies, like automatic thresholding (e.g., Otsu's method) or machine learning, would be necessary to increase accuracy.

**Surface Roughness:**

Floodwater under high winds or through fast-moving channels can become rough, increasing backscatter and causing the SAR signal to be misclassified as land. This leads to an underestimation of the true flood extent.

**Temporal Resolution Limit:**

While Sentinel-1 has a good revisit time (6-12 days over Europe), flash floods can occur and recede rapidly, sometimes within hours. The mean post-flood composite generated here may capture the maximum extent, but it does not resolve the peak inundation if it occurred between satellite passes.

**Inaccessibility of Data:**

All necessary raw Sentinel-1 and JRC data were readily available through the GEE catalog. No inaccessibility issues were encountered, which is a testament to the open data policies of Copernicus and the robustness of the GEE platform. However, the limitation of only being able to use GRD (Ground Range Detected) data, which is less geometrically precise than SLC (Single Look Complex) data for advanced InSAR analysis, restricts the scope of exploratory analysis, such as measuring ground displacement, which was not pursued here.

**Conclusion**

This project successfully implemented a Sentinel-1 change detection workflow on the Google Earth Engine platform to map the land-based inundation caused by the October 2024 DANA event in the Valencia region. By integrating the JRC Global Surface Water mask, the analysis produced a geographically constrained flood map and yielded a quantitative area estimate, demonstrating the effectiveness of SAR and cloud-based GIS for rapid disaster assessment. Future work should focus on optimizing flood detection algorithms through dynamic, localized thresholding or machine learning classification to mitigate the inherent ambiguities associated with fixed backscatter thresholds.

**Bibliography**

**References**



1.   Filipponi, F. (2019). Sentinel-1 GRD preprocessing workflow. Remote Sensing, 11(6), 766. (GIS/Satellite Tool Use)
2.   Giustarini, L., et al. (2016). Mapping flood extent from Sentinel-1 data using change detection techniques. Remote Sensing of Environment, 180, 26–42. (GIS/Satellite Tool Use)

1.   Konishi, T., et al. (2019). Detection of landslides triggered by heavy rainfall using Sentinel-1 SAR data. International Journal of Applied Earth Observation and Geoinformation, 79, 163–175.

1.   Lasaponara, R., & Masini, N. (2017). Satellite SAR and optical data for supporting forest fires detection and post-event analysis: a case study in Apulia, Italy. Natural Hazards and Earth System Sciences, 17, 981–996.
2.   Shahabi, H., et al. (2020). Flood Detection and Susceptibility Mapping Using Sentinel-1 Remote Sensing Data and a Machine Learning Approach. Remote Sensing, 12(2), 266. (GIS/Satellite Tool Use)





